In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

In [2]:
df= spark.read.csv("/FileStore/tables/gun.csv",header=True)

In [3]:
df.createOrReplaceTempView("gun")

In [4]:
df1= spark.sql("select * from gun")

In [5]:
df2= df1.repartition(250).cache()

In [6]:
display(df2)

incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
127313,2014-04-18,Florida,Fort Myers,Dunbar Street,0,0,http://www.gunviolencearchive.org/incident/127313,http://www.nbc-2.com/story/25285946/fmpd-investigating-shots-fired-at-a-home-on-palm-avenue#.U1RFQiqtMok,False,19,null,null,"Shots fired, no action (reported, no evidence found)",26.638,null,-81.853,null,null,null,null,null,null,null,null,null,http://www.nbc-2.com/story/25285946/fmpd-investigating-shots-fired-at-a-home-on-palm-avenue#.U1RFQiqtMok,78,30
174630,2014-08-15,District of Columbia,Washington,900 block of 11th Street SE,0,0,http://www.gunviolencearchive.org/incident/174630,http://www.washingtonpost.com/local/crime/dc-man-26-arrested-in-wave-of-robberies-of-businesses-since-mid-august-police-say/2014/09/06/968b3834-3579-11e4-a723-fa3895a25d02_story.html,False,1,null,null,Armed robbery with injury/death and/or evidence of DGU found,38.8793,null,-76.9915,null,null,0::26,0::Adult 18+,0::Male,0::Curtis Fogg,null,0::Unharmed,0::Subject-Suspect,http://www.washingtonpost.com/local/crime/dc-man-26-arrested-in-wave-of-robberies-of-businesses-since-mid-august-police-say/2014/09/06/968b3834-3579-11e4-a723-fa3895a25d02_story.html,null,null
122373,2014-04-05,Tennessee,Clarksville,null,0,0,http://www.gunviolencearchive.org/incident/122373,http://www.theleafchronicle.com/article/20140405/NEWS01/304050006/-1/rss,False,7,null,null,Armed robbery with injury/death and/or evidence of DGU found,36.5389,null,-87.3386,null,parking lot behind Castle Heights 1v 3p AR,null,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+,0::Male,null,null,0::Unharmed||1::Unharmed||2::Unharmed||3::Unharmed,0::Victim||1::Subject-Suspect||2::Subject-Suspect||3::Subject-Suspect,http://www.theleafchronicle.com/article/20140405/NEWS01/304050006/-1/rss,67,22
112086,2014-03-04,California,Paramount,7200 block of Richfield Street,1,0,http://www.gunviolencearchive.org/incident/112086,http://homicide.latimes.com/post/alexandro-israel-carreon/,False,40,null,null,"Shot - Dead (murder, accidental, suicide)",33.9017,null,-118.173,null,null,0:22,0:Adult 18+,0:Male,0:Alexandro Israel Carreon,null,0:Killed,0:Victim,http://losangeles.cbslocal.com/2014/03/04/homicide-detectives-investigate-fatal-shooting-in-paramount/|http://homicide.latimes.com/post/alexandro-israel-carreon/,63,33
134210,2014-05-12,Ohio,Cincinnati,934 Chateau,0,2,http://www.gunviolencearchive.org/incident/134210,http://www.local12.com/news/features/top-stories/stories/two-women-shot-east-price-hill-11580.shtml#.U3DXVF5RrHg,False,1,null,null,Shot - Wounded/Injured,39.1079,null,-84.5598,null,Cedar Hill apartment complex - 2 women shot and injured outside apt.,null,0::Adult 18+||1::Adult 18+,0::Female||1::Female,null,null,0::Injured||1::Injured,0::Victim||1::Victim,http://www.local12.com/news/features/top-stories/stories/two-women-shot-east-price-hill-11580.shtml#.U3DXVF5RrHg,29,8
571282,2014-06-07,New York,Bronx,789 Fairmount Avenue,0,1,http://www.gunviolencearchive.org/incident/571282,http://nypdnews.com/2016/06/wanted-robbery-bronx-69/,False,15,null,null,Shot - Wounded/Injured,40.8427,null,-73.8888,null,"48th precinct, shot in stomach",0::32||1::45,0::Adult 18+||1::Adult 18+,0::Male||1::Male,"""1::Randolph """"Dane Buju"""" Phillip""",null,"0::Injured||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://nypdnews.com/2016/06/wanted-robbery-bronx-69/,79,33
153408,2014-06-30,Massachusetts,Boston,73 Edgewater Drive,1,0,http://www.gunviolencearchive.org/incident/153408,http://bpdnews.com/news/2014/7/2/update-victim-identified-in-death-investigation-in-the-area-of-73-edgewater-d

In [7]:
df2.select("state","n_killed","city_or_county","n_injured") \
   .filter(F.col("n_killed")>=1)\
   .filter(F.col("n_injured")>=1)\
   .filter(F.col("state")=="Florida")\
   .count()

Out[48]: 602

In [8]:
df3=df2.withColumn("Year",F.year(F.col("date").cast("date")))

In [9]:
df6=df3.groupBy("Year")\
       .agg(F.sum(F.col("n_killed")).alias("SUM"))\
       .select("Year","SUM")\
       .dropna()\
       .orderBy("Year")
       

In [10]:
display(df6)

Year,SUM
2013,317.0
2014,12557.0
2015,13484.0
2016,15066.0
2017,15511.0
2018,3533.0


In [11]:
df7=df3.groupBy("State","Year")\
       .agg(F.sum(F.col("n_killed")).alias("SUM"))\
       .select("Year","SUM")\
       .dropna()\
       .orderBy("Year")\
       .filter(F.col("state")=="Florida")

In [12]:
display(df7)

Year,SUM
2013,22.0
2014,873.0
2015,796.0
2016,1001.0
2017,950.0
2018,267.0


In [13]:
df8=df3.groupBy("state")\
       .agg(F.sum(F.col("n_killed")).alias("Killed"),(F.sum(F.col("n_injured")).alias("Injured")))\
       .select("state","Killed","Injured")\
       .dropna()\
       .orderBy(F.col("Injured").desc())

state,Killed,Injured
Illinois,3409.0,13514.0
California,5562.0,7644.0
Florida,3909.0,7072.0
Texas,5046.0,6106.0
Ohio,2508.0,5703.0
New York,1705.0,5059.0
Pennsylvania,2395.0,5056.0
North Carolina,2225.0,4629.0
Tennessee,1824.0,4478.0
Louisiana,2179.0,4398.0


In [15]:
df9=df8.withColumn("Total",F.col("Killed")+F.col("Injured")).orderBy(F.col("Total").desc())
display(df9.select("state","Total").limit(20))


state,Total
Illinois,16923.0
California,13206.0
Texas,11152.0
Florida,10981.0
Ohio,8211.0
Pennsylvania,7451.0
North Carolina,6854.0
New York,6764.0
Louisiana,6577.0
Georgia,6512.0


In [16]:
display(df3.limit(10))

incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,Year
127313,2014-04-18,Florida,Fort Myers,Dunbar Street,0,0,http://www.gunviolencearchive.org/incident/127313,http://www.nbc-2.com/story/25285946/fmpd-investigating-shots-fired-at-a-home-on-palm-avenue#.U1RFQiqtMok,False,19,null,null,"Shots fired, no action (reported, no evidence found)",26.638,null,-81.853,null,null,null,null,null,null,null,null,null,http://www.nbc-2.com/story/25285946/fmpd-investigating-shots-fired-at-a-home-on-palm-avenue#.U1RFQiqtMok,78,30,2014
174630,2014-08-15,District of Columbia,Washington,900 block of 11th Street SE,0,0,http://www.gunviolencearchive.org/incident/174630,http://www.washingtonpost.com/local/crime/dc-man-26-arrested-in-wave-of-robberies-of-businesses-since-mid-august-police-say/2014/09/06/968b3834-3579-11e4-a723-fa3895a25d02_story.html,False,1,null,null,Armed robbery with injury/death and/or evidence of DGU found,38.8793,null,-76.9915,null,null,0::26,0::Adult 18+,0::Male,0::Curtis Fogg,null,0::Unharmed,0::Subject-Suspect,http://www.washingtonpost.com/local/crime/dc-man-26-arrested-in-wave-of-robberies-of-businesses-since-mid-august-police-say/2014/09/06/968b3834-3579-11e4-a723-fa3895a25d02_story.html,null,null,2014
122373,2014-04-05,Tennessee,Clarksville,null,0,0,http://www.gunviolencearchive.org/incident/122373,http://www.theleafchronicle.com/article/20140405/NEWS01/304050006/-1/rss,False,7,null,null,Armed robbery with injury/death and/or evidence of DGU found,36.5389,null,-87.3386,null,parking lot behind Castle Heights 1v 3p AR,null,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+,0::Male,null,null,0::Unharmed||1::Unharmed||2::Unharmed||3::Unharmed,0::Victim||1::Subject-Suspect||2::Subject-Suspect||3::Subject-Suspect,http://www.theleafchronicle.com/article/20140405/NEWS01/304050006/-1/rss,67,22,2014
112086,2014-03-04,California,Paramount,7200 block of Richfield Street,1,0,http://www.gunviolencearchive.org/incident/112086,http://homicide.latimes.com/post/alexandro-israel-carreon/,False,40,null,null,"Shot - Dead (murder, accidental, suicide)",33.9017,null,-118.173,null,null,0:22,0:Adult 18+,0:Male,0:Alexandro Israel Carreon,null,0:Killed,0:Victim,http://losangeles.cbslocal.com/2014/03/04/homicide-detectives-investigate-fatal-shooting-in-paramount/|http://homicide.latimes.com/post/alexandro-israel-carreon/,63,33,2014
134210,2014-05-12,Ohio,Cincinnati,934 Chateau,0,2,http://www.gunviolencearchive.org/incident/134210,http://www.local12.com/news/features/top-stories/stories/two-women-shot-east-price-hill-11580.shtml#.U3DXVF5RrHg,False,1,null,null,Shot - Wounded/Injured,39.1079,null,-84.5598,null,Cedar Hill apartment complex - 2 women shot and injured outside apt.,null,0::Adult 18+||1::Adult 18+,0::Female||1::Female,null,null,0::Injured||1::Injured,0::Victim||1::Victim,http://www.local12.com/news/features/top-stories/stories/two-women-shot-east-price-hill-11580.shtml#.U3DXVF5RrHg,29,8,2014
571282,2014-06-07,New York,Bronx,789 Fairmount Avenue,0,1,http://www.gunviolencearchive.org/incident/571282,http://nypdnews.com/2016/06/wanted-robbery-bronx-69/,False,15,null,null,Shot - Wounded/Injured,40.8427,null,-73.8888,null,"48th precinct, shot in stomach",0::32||1::45,0::Adult 18+||1::Adult 18+,0::Male||1::Male,"""1::Randolph """"Dane Buju"""" Phillip""",null,"0::Injured||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://nypdnews.com/2016/06/wanted-robbery-bronx-69/,79,33,2014
153408,2014-06-30,Massachusetts,Boston,73 Edgewater Drive,1,0,http://www.gunviolencearchive.org/incident/153408,http://bpdnews.com/news/2014/7/2/update-victim-identified-in-death-investig

In [17]:
dfu=df3.select("state","Year","n_killed","n_injured","incident_characteristics")\
   .filter(F.col("incident_characteristics").rlike('Mass'))
dfuu= dfu.groupBy("state")\
         .agg(F.sum(F.col("n_killed")).alias("Killed"),F.sum(F.col("n_injured")).alias("Injured"))\
         .withColumn("Total",F.col("Killed")+F.col("Injured"))\
         .orderBy(F.col("Total").desc())
display(dfuu.limit(20))
  

state,Killed,Injured,Total
California,202.0,780.0,982.0
Illinois,110.0,611.0,721.0
Florida,194.0,524.0,718.0
Texas,192.0,393.0,585.0
New York,53.0,323.0,376.0
Louisiana,58.0,289.0,347.0
Georgia,80.0,260.0,340.0
Ohio,71.0,242.0,313.0
Pennsylvania,58.0,254.0,312.0
Tennessee,50.0,248.0,298.0


In [18]:
df3.filter(F.col("incident_characteristics").rlike("suicide")).count()

Out[172]: 55004

In [19]:
dfn=df3.select("state","Year","n_killed","n_injured","incident_characteristics")\
   .filter(F.col("incident_characteristics").rlike('suicide'))
dfnn= dfn.groupBy("state")\
         .agg(F.sum(F.col("n_killed")).alias("Killed"),F.sum(F.col("n_injured")).alias("Injured"))\
         .withColumn("Total",F.col("Killed")+F.col("Injured"))\
         .orderBy(F.col("Total").desc())
display(dfnn.limit(20))

state,Killed,Injured,Total
California,5511.0,1361.0,6872.0
Texas,4976.0,1050.0,6026.0
Florida,3851.0,1020.0,4871.0
Illinois,3382.0,1029.0,4411.0
Ohio,2481.0,553.0,3034.0
Georgia,2441.0,587.0,3028.0
Pennsylvania,2370.0,518.0,2888.0
North Carolina,2201.0,457.0,2658.0
Louisiana,2152.0,474.0,2626.0
Missouri,2122.0,483.0,2605.0


In [20]:
dfn=df3.select("state","Year","n_killed","n_injured","incident_characteristics")\
   .filter(F.col("incident_characteristics").rlike('accidental'))
dfnn= dfn.groupBy("state")\
         .agg(F.sum(F.col("n_killed")).alias("Killed"),F.sum(F.col("n_injured")).alias("Injured"))\
         .withColumn("Total",F.col("Killed")+F.col("Injured"))\
         .orderBy(F.col("Total").desc())
display(dfnn.limit(20))

state,Killed,Injured,Total
California,5508.0,1305.0,6813.0
Texas,4974.0,947.0,5921.0
Florida,3849.0,914.0,4763.0
Illinois,3382.0,935.0,4317.0
Ohio,2480.0,481.0,2961.0
Georgia,2440.0,515.0,2955.0
Pennsylvania,2369.0,458.0,2827.0
North Carolina,2200.0,407.0,2607.0
Louisiana,2151.0,443.0,2594.0
Missouri,2121.0,447.0,2568.0


In [21]:
dfp= spark.read.csv("/FileStore/tables/statepop.csv",header=True)
display(dfp)

State,2018 Population
California,39776830
Texas,28704330
Florida,21312211
New York,19862512
Pennsylvania,12823989
Illinois,12768320
Ohio,11694664
Georgia,10545138
North Carolina,10390149
Michigan,9991177


In [22]:
dfnew= df3.join(dfp, (df3.state==dfp.State)).drop(dfp.State)


In [23]:
display(dfnew)

incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,Year,2018 Population
127313,2014-04-18,Florida,Fort Myers,Dunbar Street,0,0,http://www.gunviolencearchive.org/incident/127313,http://www.nbc-2.com/story/25285946/fmpd-investigating-shots-fired-at-a-home-on-palm-avenue#.U1RFQiqtMok,False,19,null,null,"Shots fired, no action (reported, no evidence found)",26.638,null,-81.853,null,null,null,null,null,null,null,null,null,http://www.nbc-2.com/story/25285946/fmpd-investigating-shots-fired-at-a-home-on-palm-avenue#.U1RFQiqtMok,78,30,2014,21312211
174630,2014-08-15,District of Columbia,Washington,900 block of 11th Street SE,0,0,http://www.gunviolencearchive.org/incident/174630,http://www.washingtonpost.com/local/crime/dc-man-26-arrested-in-wave-of-robberies-of-businesses-since-mid-august-police-say/2014/09/06/968b3834-3579-11e4-a723-fa3895a25d02_story.html,False,1,null,null,Armed robbery with injury/death and/or evidence of DGU found,38.8793,null,-76.9915,null,null,0::26,0::Adult 18+,0::Male,0::Curtis Fogg,null,0::Unharmed,0::Subject-Suspect,http://www.washingtonpost.com/local/crime/dc-man-26-arrested-in-wave-of-robberies-of-businesses-since-mid-august-police-say/2014/09/06/968b3834-3579-11e4-a723-fa3895a25d02_story.html,null,null,2014,703608
122373,2014-04-05,Tennessee,Clarksville,null,0,0,http://www.gunviolencearchive.org/incident/122373,http://www.theleafchronicle.com/article/20140405/NEWS01/304050006/-1/rss,False,7,null,null,Armed robbery with injury/death and/or evidence of DGU found,36.5389,null,-87.3386,null,parking lot behind Castle Heights 1v 3p AR,null,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+,0::Male,null,null,0::Unharmed||1::Unharmed||2::Unharmed||3::Unharmed,0::Victim||1::Subject-Suspect||2::Subject-Suspect||3::Subject-Suspect,http://www.theleafchronicle.com/article/20140405/NEWS01/304050006/-1/rss,67,22,2014,6782564
112086,2014-03-04,California,Paramount,7200 block of Richfield Street,1,0,http://www.gunviolencearchive.org/incident/112086,http://homicide.latimes.com/post/alexandro-israel-carreon/,False,40,null,null,"Shot - Dead (murder, accidental, suicide)",33.9017,null,-118.173,null,null,0:22,0:Adult 18+,0:Male,0:Alexandro Israel Carreon,null,0:Killed,0:Victim,http://losangeles.cbslocal.com/2014/03/04/homicide-detectives-investigate-fatal-shooting-in-paramount/|http://homicide.latimes.com/post/alexandro-israel-carreon/,63,33,2014,39776830
134210,2014-05-12,Ohio,Cincinnati,934 Chateau,0,2,http://www.gunviolencearchive.org/incident/134210,http://www.local12.com/news/features/top-stories/stories/two-women-shot-east-price-hill-11580.shtml#.U3DXVF5RrHg,False,1,null,null,Shot - Wounded/Injured,39.1079,null,-84.5598,null,Cedar Hill apartment complex - 2 women shot and injured outside apt.,null,0::Adult 18+||1::Adult 18+,0::Female||1::Female,null,null,0::Injured||1::Injured,0::Victim||1::Victim,http://www.local12.com/news/features/top-stories/stories/two-women-shot-east-price-hill-11580.shtml#.U3DXVF5RrHg,29,8,2014,11694664
571282,2014-06-07,New York,Bronx,789 Fairmount Avenue,0,1,http://www.gunviolencearchive.org/incident/571282,http://nypdnews.com/2016/06/wanted-robbery-bronx-69/,False,15,null,null,Shot - Wounded/Injured,40.8427,null,-73.8888,null,"48th precinct, shot in stomach",0::32||1::45,0::Adult 18+||1::Adult 18+,0::Male||1::Male,"""1::Randolph """"Dane Buju"""" Phillip""",null,"0::Injured||1::Unharmed, Arrested",0::Victim||1::Subject-Suspect,http://nypdnews.com/2016/06/wanted-robbery-bronx-69/,79,33,2014,19862512
153408,2014-06-30,Massachusetts,Boston,73 Edgewater Drive,1,0,http://www.gunviolencearchive.org/incident/153408,http://b

In [24]:
dfnew.withColumn("Capita",F.col("2018 Population")/F.col(""))

In [25]:
'''dfu=df3.select("state","Year","n_killed","n_injured","incident_characteristics")\
   .filter(F.col("incident_characteristics").rlike('Mass'))'''
dfuu= dfnew.groupBy("state","2018 Population")\
         .agg(F.sum(F.col("n_killed")).alias("Killed"),F.sum(F.col("n_injured")).alias("Injured"))\
         .withColumn("Capita",F.col("2018 Population")/(F.col("Killed")+F.col("Injured")))\
         .orderBy(F.col("Capita").desc())
display(dfuu.limit(20))
  

state,2018 Population,Killed,Injured,Capita
Hawaii,1426393,63.0,85.0,9637.79054054054
New Hampshire,1350575,88.0,144.0,5821.443965517241
Idaho,1753860,166.0,149.0,5567.809523809524
Maine,1341582,112.0,132.0,5498.286885245901
Vermont,623960,57.0,73.0,4799.692307692308
Utah,3159345,282.0,402.0,4618.925438596491
Wyoming,573720,73.0,52.0,4589.76
South Dakota,877790,90.0,114.0,4302.892156862745
Minnesota,5628162,461.0,916.0,4087.263616557734
North Dakota,755238,69.0,130.0,3795.165829145729
